In [40]:
import datetime
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import widgets

 Import the data‘Flights dataset.csv’

In [41]:
df = pd.read_csv( 'Flights dataset.csv')

Set the range of days in December

In [42]:
day_of_month = widgets.IntSlider(
    value=1.0,
    min=1.0,
    max=31.0,
    step=1.0,
    description='Day:',
    continuous_update=False
)

Set option box

In [43]:
use_date = widgets.Checkbox(
    description='December 2009: ',
    value=True,
)

In [44]:
container = widgets.HBox(children=[use_date, day_of_month])

Set the drop-down box about the origin

In [45]:
origin = widgets.Dropdown(
    options=list(df['ORIGIN'].unique()),
    value='ATL',
    description='Origin Airport:',
)

set trace1 for Cancelled Rate and set trace1 for Departure Delay Rate

In [46]:
dff=df.groupby('CARRIER').CANCELLED.mean().round(2)
trace1 = go.Scatter(x=dff.index,y=dff.values, name='Cancelled Rate')

df['DEP_DELAY'] = np.where(df['DEP_DELAY']>0,1,0)
dff2=df.groupby('CARRIER').DEP_DELAY.mean().round(2)

trace2 = go.Scatter(x=dff2.index,y=dff2.values, name='Departure Delay Rate')
g = go.FigureWidget(data=[trace1, trace2],
                    layout=go.Layout(
                        title=dict(
                            text='Cancelled Rate & Departure Delay Rate of U.S. flights in December 2009',
                            y=0.85,
                            x=0.435,
                            xanchor= 'center',
                            yanchor= 'top'),
                        barmode='overlay'
                    ))

Define response function and use observe function to control the properties.
When the value of these properties changes, the response function will be recalled
and a set of data will be imported as the response variable.

In [47]:
def validate():
    if origin.value in df['ORIGIN'].unique():
        return True
    else:
        return False


def response(change):
    if validate():
        if use_date.value:
            filter_list = [i and j for i, j in
                           zip(df['DAY_OF_MONTH'] == day_of_month.value,
                               df['ORIGIN'] == origin.value)]
            temp_df = df[filter_list]
        else:
            filter_list = [i for i in df['ORIGIN'] == origin.value]
            temp_df = df[filter_list]
        x1 = temp_df.groupby('CARRIER').CANCELLED.mean().round(2).index
        y1 = temp_df.groupby('CARRIER').CANCELLED.mean().round(2).values
        temp_df['DEP_DELAY'] = np.where(temp_df['DEP_DELAY']>0,1,0)
        x2 = temp_df.groupby('CARRIER').DEP_DELAY.mean().round(2).index
        y2 = temp_df.groupby('CARRIER').DEP_DELAY.mean().round(2).values
        with g.batch_update():
            g.data[0].x = x1
            g.data[0].y = y1
            g.data[1].x = x2
            g.data[1].y = y2
            g.layout.xaxis.title = 'CARRIER'
            g.layout.yaxis.title = 'RATE'

origin.observe(response, names="value")
day_of_month.observe(response, names="value")
use_date.observe(response, names="value")

Show the final interactive diagram

In [48]:
container2 = widgets.HBox([origin])
widgets.VBox([container,
              container2,
              g])